# ASR Commands (Keyword Spotting) — Production-grade HF baseline

This notebook trains a **keyword spotting / speech command classification** model on TensorFlow's **mini_speech_commands** dataset.

## Design goals
- Minimal custom modeling code (rely on mature ecosystem).
- Deterministic splits + reproducibility.
- Clean artifact export (model + processor + labels + metrics).

## Architecture
1) `data_ingestion/asr_commands` downloads+verifies dataset into `.cache/`
2) Dataset is loaded as `{audio, label}` with `datasets.Audio(sampling_rate=16_000)`
3) Model is fine-tuned via `transformers.Trainer` using a pretrained speech encoder (`HuBERT` / `Wav2Vec2`)
4) Evaluation exports accuracy, macro-F1, and confusion matrix into `outputs/asr_commands/`

In [ ]:
%pip uninstall -y torch torchvision torchaudio torchcodec -q
%pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 torchcodec -q

In [ ]:
# Install runtime deps (Colab-friendly). Restart runtime/kernel if you upgrade torch.
!pip -q install --upgrade "transformers>=4.38" "datasets[audio]>=2.17" "evaluate>=0.4" "accelerate>=0.26" "scikit-learn>=1.3" "soundfile>=0.12"

In [ ]:
# Colab convenience: clone repo if needed
from pathlib import Path

if not Path("pyproject.toml").exists():
    if not Path("pjatk_zum").exists():
        !git clone https://github.com/beep1000101/pjatk_zum.git
    else:
        print("Repo folder already present: pjatk_zum")
else:
    print("Already in repo root (pyproject.toml found)")

In [ ]:
# Colab convenience: cd into repo folder if we cloned it
from pathlib import Path

if Path("pyproject.toml").exists():
    print("Already in repo root")
elif Path("pjatk_zum").exists():
    %cd pjatk_zum
else:
    raise FileNotFoundError("Could not find repo root (pyproject.toml) or ./pjatk_zum")

In [ ]:
# Ingest mini_speech_commands into the repo cache
!python data_ingestion/asr_commands/run.py

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import json
import os
import random
import sys

import numpy as np
import pandas as pd

import torch

from datasets import Audio, Dataset, DatasetDict
import evaluate
from transformers import (
    AutoModelForAudioClassification,
    AutoProcessor,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback,
 )


def find_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for p in [cur, *cur.parents]:
        if (p / "data_ingestion").exists() and (p / "utils").exists():
            return p
    return cur


ROOT = find_repo_root(Path())
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from utils.paths import CACHE_PATH  # noqa: E402

# Reproducibility
SEED = 42
set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

CACHE = CACHE_PATH
PIPELINE_CACHE = CACHE / "asr_commands"
RAW_DIR = PIPELINE_CACHE / "raw"
OUTPUTS_DIR = ROOT / "outputs" / "asr_commands"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
# Ensure dataset is present in cache (runs ingestion if needed)
mini_root = RAW_DIR / "mini_speech_commands"
if not mini_root.exists():
    print("mini_speech_commands not found in cache; running ingestion...")
    !python data_ingestion/asr_commands/run.py

assert mini_root.exists(), f"Missing: {mini_root}"

labels_path = PIPELINE_CACHE / "labels.json"
if labels_path.exists():
    labels = json.loads(labels_path.read_text(encoding="utf-8"))["labels"]
else:
    # Fallback: infer from directories
    labels = sorted([p.name for p in mini_root.iterdir() if p.is_dir()])

label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for lbl, i in label2id.items()}

labels, len(labels)

['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

In [ ]:
# Build an index of audio files -> labels
rows = []
for lbl in labels:
    for wav_path in sorted((mini_root / lbl).glob("*.wav")):
        rows.append({"audio": str(wav_path), "label_str": lbl, "label": int(label2id[lbl])})

df = pd.DataFrame(rows)
df["label_str"].value_counts()

label
down     1000
go       1000
left     1000
no       1000
right    1000
stop     1000
up       1000
yes      1000
Name: count, dtype: int64

In [ ]:
# Deterministic stratified split 80/10/10 using 🤗 Datasets
# NOTE: `stratify_by_column` requires a `ClassLabel` feature.
from datasets import ClassLabel

full_ds = Dataset.from_pandas(df, preserve_index=False)
full_ds = full_ds.cast_column("label", ClassLabel(names=labels))

# 80% train, 20% temp (val+test)
tmp = full_ds.train_test_split(test_size=0.2, seed=SEED, stratify_by_column="label")
train_ds_raw = tmp["train"]
temp_ds = tmp["test"]

# Split temp into 50/50 -> 10% val, 10% test
tmp2 = temp_ds.train_test_split(test_size=0.5, seed=SEED, stratify_by_column="label")
val_ds_raw = tmp2["train"]
test_ds_raw = tmp2["test"]

ds = DatasetDict({"train": train_ds_raw, "val": val_ds_raw, "test": test_ds_raw})
{k: len(v) for k, v in ds.items()}

{'train': 6400, 'val': 800, 'test': 800}

In [ ]:
# Persist splits for reuse (stable artifact for later stage-first scripts)
(OUTPUTS_DIR / "preprocessing").mkdir(parents=True, exist_ok=True)
splits_path = OUTPUTS_DIR / "preprocessing" / "splits.json"

def _to_records(dset: Dataset) -> list[dict]:
    # Keep the same schema used elsewhere in the repo
    return [{"path": ex["audio"], "label": id2label[int(ex["label"]) ]} for ex in dset]

payload = {
    "labels": labels,
    "label2id": label2id,
    "splits": {
        "train": _to_records(ds["train"]),
        "val": _to_records(ds["val"]),
        "test": _to_records(ds["test"]),
    },
}
splits_path.write_text(json.dumps(payload, indent=2), encoding="utf-8")

splits_path

PosixPath('/home/mateusz/dev/pjatk_zum/outputs/asr_commands/preprocessing/splits.json')

In [ ]:
# HF audio pipeline: decode/resample -> feature_extractor -> padded batches
# Model choice: HuBERT encoder pretrained + KWS-oriented finetuning (SUPERB)
MODEL_NAME = "superb/hubert-base-superb-ks"

from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

# Decode audio lazily; `datasets` uses soundfile under the hood
ds_audio = ds.cast_column("audio", Audio(sampling_rate=16_000))

def preprocess_batch(batch):
    audio = batch["audio"]
    out = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"])
    batch["input_values"] = out["input_values"][0]
    if "attention_mask" in out:
        batch["attention_mask"] = out["attention_mask"][0]
    return batch

ds_proc = ds_audio.map(preprocess_batch, remove_columns=["audio"])

data_collator = DataCollatorWithPadding(processor=feature_extractor, padding=True)

ds_proc["train"][0].keys()

(6400, 800, 800)

In [ ]:
# Trainer setup + fine-tuning
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels_np)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=labels_np, average="macro")["f1"]
    return {"accuracy": float(acc), "f1_macro": float(f1)}

model = AutoModelForAudioClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

# Stability on small datasets: freeze encoder features (optional but recommended)
if hasattr(model, "freeze_feature_encoder"):
    model.freeze_feature_encoder()

run_dir = OUTPUTS_DIR / "hf_training"
run_dir.mkdir(parents=True, exist_ok=True)

training_args = TrainingArguments(
    output_dir=str(run_dir),
    seed=SEED,
    data_seed=SEED,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    num_train_epochs=12,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to=[],
 )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_proc["train"],
    eval_dataset=ds_proc["val"],
    data_collator=data_collator,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

train_out = trainer.train()
train_out

ImportError: TorchCodec is required for load_with_torchcodec. Please install torchcodec to use this function.

In [ ]:
# Save best model + processor + labels (portable inference package)
best_dir = OUTPUTS_DIR / "best_hf"
best_dir.mkdir(parents=True, exist_ok=True)

trainer.save_model(str(best_dir))
processor.save_pretrained(str(best_dir))
(best_dir / "labels.json").write_text(json.dumps({"labels": labels, "label2id": label2id}, indent=2), encoding="utf-8")

best_dir

In [ ]:
# Export training log history (JSON)
train_metrics_path = OUTPUTS_DIR / "training" / "metrics.json"
train_metrics_path.parent.mkdir(parents=True, exist_ok=True)

history = trainer.state.log_history
train_metrics_path.write_text(
    json.dumps({"pipeline": "asr_commands", "framework": "hf_trainer", "history": history}, indent=2),
    encoding="utf-8",
)

train_metrics_path

In [ ]:
# Evaluate on test split
test_metrics = trainer.evaluate(ds_proc["test"])
test_metrics

In [ ]:
# Confusion matrix + full metrics export
from sklearn.metrics import confusion_matrix, classification_report

pred_out = trainer.predict(ds_proc["test"])
logits = pred_out.predictions
y_true = pred_out.label_ids
y_pred = np.argmax(logits, axis=-1)

cm = confusion_matrix(y_true, y_pred, labels=list(range(len(labels))))
report = classification_report(
    y_true,
    y_pred,
    labels=list(range(len(labels))),
    target_names=labels,
    output_dict=True,
    zero_division=0,
)

eval_metrics = {
    "pipeline": "asr_commands",
    "num_test": int(len(y_true)),
    "accuracy": float(test_metrics.get("eval_accuracy", test_metrics.get("accuracy", 0.0))),
    "f1_macro": float(test_metrics.get("eval_f1_macro", test_metrics.get("f1_macro", 0.0))),
    "labels": labels,
    "label2id": label2id,
    "confusion_matrix": cm.tolist(),
    "classification_report": report,
}

eval_dir = OUTPUTS_DIR / "evaluation"
eval_dir.mkdir(parents=True, exist_ok=True)
eval_metrics_path = eval_dir / "metrics.json"
eval_metrics_path.write_text(json.dumps(eval_metrics, indent=2), encoding="utf-8")

eval_metrics_path

In [ ]:
# (Optional) Zip outputs for sharing (e.g., Colab -> download)
!zip -r asr_commands_outputs.zip outputs/asr_commands